In [1]:
import pandas as pd
import http.client
import requests
import urllib.parse
import json
import time
import math
import re
import unicodedata

In [24]:
df = pd.read_excel(~path to file)

In [25]:
def normalize_string(s):
    """
    Normalizes a string by removing accents and diacritics.
    Converts the string to lowercase.
    """
    if not s:
        return ''
    # Normalize string to NFD (Normalization Form Decomposition)
    normalized = unicodedata.normalize('NFD', s)
    # Filter out combining characters (diacritics)
    without_diacritics = ''.join(
        c for c in normalized if unicodedata.category(c) != 'Mn'
    )
    # Convert to lowercase
    return without_diacritics.lower()


In [26]:
def clean_address(address, city=None, province=None, country=None):
    """
    Cleans and standardizes an address string for geocoding.
    Optionally appends city, province, and country if they are not already present.

    :param address: Original address string.
    :param city: City name.
    :param province: Province or state name.
    :param country: Country name.
    :return: Cleaned address string.
    """
    # Remove leading and trailing whitespace
    address = address.strip()

    # Replace line breaks and tabs with a comma
    address = re.sub(r'[\r\n\t]+', ', ', address)

    # Remove multiple commas and replace with a single comma
    address = re.sub(r',\s*,+', ', ', address)

    # Replace multiple spaces with a single space
    address = re.sub(r'\s+', ' ', address)

    # Remove unwanted special characters but keep essential punctuation
    address = re.sub(r'[^\w\s,.-/]', '', address)

    # Normalize the address for comparison
    address_normalized = normalize_string(address)

    # Append city if not in address and provided
    if city and city.strip():
        city_clean = city.strip()
        city_normalized = normalize_string(city_clean)
        if city_normalized not in address_normalized:
            address += f', {city_clean}'

    # Append province if not in address and provided
    if province and province.strip():
        province_clean = province.strip()
        province_normalized = normalize_string(province_clean)
        if province_normalized not in address_normalized:
            address += f', {province_clean}'

    # Append country if not in address and provided
    if country and country.strip():
        country_clean = country.strip()
        country_normalized = normalize_string(country_clean)
        if country_normalized not in address_normalized:
            address += f', {country_clean}'

    # Remove multiple commas again after appending components
    address = re.sub(r',\s*,+', ', ', address)

    # Return the cleaned address
    return address


In [27]:
def geocode_address_google(search_term, api_key):
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'address': search_term,
        'key': api_key,
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        result = response.json()

        if result['status'] == 'OK':
            # Iterate over results to find the best match
            for res in result['results']:
                location_type = res['geometry']['location_type']
                result_types = res['types']

                # Accept a broader range of result types
                valid_location_types = ['ROOFTOP', 'RANGE_INTERPOLATED', 'GEOMETRIC_CENTER', 'APPROXIMATE']
                valid_result_types = [
                    'street_address',
                    'premise',           # Named locations (buildings, etc.)
                    'establishment',     # Places of business, organizations
                    'point_of_interest', # Notable locations
                    'subpremise'        # Units within a premise
                ]

                if (location_type in valid_location_types and 
                    any(type in result_types for type in valid_result_types)):
                    location = res['geometry']['location']
                    formatted_address = res['formatted_address']
                    address_components = res['address_components']
                    
                    # Add confidence level based on result type
                    confidence = {
                        'ROOFTOP': 'High',
                        'RANGE_INTERPOLATED': 'Medium-High',
                        'GEOMETRIC_CENTER': 'Medium',
                        'APPROXIMATE': 'Low'
                    }.get(location_type, 'Unknown')

                    print(f"Found location with confidence level: {confidence}")
                    print(f"Result types: {', '.join(result_types)}")
                    
                    return location, formatted_address, address_components, location_type

            # If no result matched our criteria, take the first result if available
            if result['results']:
                res = result['results'][0]
                location = res['geometry']['location']
                formatted_address = res['formatted_address']
                address_components = res['address_components']
                location_type = res['geometry']['location_type']
                
                print(f"Using best available result (types: {', '.join(res['types'])})")
                return location, formatted_address, address_components, location_type

            print(f"No suitable result found for search term '{search_term}'")
            return None, None, None, None
        else:
            print(f"Error in geocoding search term '{search_term}': {result['status']}")
            return None, None, None, None
    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed for search term '{search_term}': {e}")
        return None, None, None, None

In [28]:
def reverse_geocode_google(lat, lon, api_key):
    """
    Reverse geocodes latitude and longitude to a formatted address, including businesses.

    :param lat: Latitude of the location.
    :param lon: Longitude of the location.
    :param api_key: Google Maps Geocoding API key.
    :return: Tuple containing (formatted_address, address_components, location_type) or (None, None, None) if failed.
    """
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {
        'latlng': f'{lat},{lon}',
        'key': api_key,
        'language': 'en', 
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        result = response.json()

        if result['status'] == 'OK':
            # Define priority criteria
            priority_location_types = ['ROOFTOP', 'RANGE_INTERPOLATED']
            priority_result_types = ['street_address', 'premise', 'point_of_interest', 'establishment']

            # Iterate through the results to find the best match
            for res in result['results']:
                location_type = res['geometry']['location_type']
                result_types = res['types']

                # Debugging: Print the types of each result
                print(f"Reverse Geocode Result Types: {result_types}")

                # Check if the result meets the priority criteria
                if (location_type in priority_location_types) and (any(rt in priority_result_types for rt in result_types)):
                    formatted_address = res['formatted_address']
                    address_components = res['address_components']
                    return formatted_address, address_components, location_type

            # Fallback: Use the first result if no high-accuracy match is found
            first_result = result['results'][0]
            formatted_address = first_result['formatted_address']
            address_components = first_result['address_components']
            location_type = first_result['geometry']['location_type']
            print(f"No high-accuracy result found. Using first result: {formatted_address}")
            return formatted_address, address_components, location_type
        else:
            print(f"Error in reverse geocoding coordinates {lat},{lon}: {result['status']}")
            return None, None, None
    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed for coordinates '{lat},{lon}': {e}")
        return None, None, None


In [29]:
def process_dataframe(df, api_key):
    for index, row in df.iterrows():
        company_name = str(row.get('Company name', '')).strip()
        site_address = str(row.get('Physical address of company', '')).strip()
        city = str(row.get('City where company is located', '')).strip()
        province = str(row.get('District and county', '')).strip()
        country = str(row.get('Country', '')).strip()

        location = formatted_address = components = location_type = None

        # Try geocoding by company name first if available
        if company_name and company_name.lower() != 'nan':
            # Combine company name with city/province/country for better accuracy
            company_search = f"{company_name}, {city}, {province}, {country}".strip(', ')
            location, formatted_address, components, location_type = geocode_address_google(company_search, api_key)
            
            if location:
                print(f"Index {index}: Successfully geocoded company '{company_name}' to coordinates "
                      f"({location['lat']}, {location['lng']}) with location type '{location_type}'")

        # If company name geocoding failed or wasn't possible, try address
        if not location and site_address and site_address.lower() != 'nan':
            cleaned_address = clean_address(site_address, city=city, province=province, country=country)
            location, formatted_address, components, location_type = geocode_address_google(cleaned_address, api_key)
            
            if location:
                print(f"Index {index}: Geocoded address '{cleaned_address}' for company '{company_name}' "
                      f"to coordinates ({location['lat']}, {location['lng']}) with location type '{location_type}'")
            else:
                print(f"Index {index}: No acceptable location found for company '{company_name}' "
                      f"using address '{cleaned_address}'")

        # Update DataFrame if either method succeeded
        if location:
            df.at[index, 'Latitude'] = location['lat']
            df.at[index, 'Longitude'] = location['lng']
            
            # Extract and update components
            city_found = None
            province_found = None
            country_found = None

            for component in components:
                if 'locality' in component['types']:
                    city_found = component['long_name']
                elif 'administrative_area_level_1' in component['types']:
                    province_found = component['long_name']
                elif 'country' in component['types']:
                    country_found = component['long_name']

            # Only update if fields are empty
            if not str(row.get('City', '')).strip():
                df.at[index, 'City'] = city_found
            if not str(row.get('Province', '')).strip():
                df.at[index, 'Province'] = province_found
            if not str(row.get('Country', '')).strip():
                df.at[index, 'Country'] = country_found

            df.at[index, 'Location Type'] = location_type
            df.at[index, 'Geocode result'] = 'Geocoded.'
            df.at[index, 'Geocode method'] = 'Company name' if company_name else 'Address'
            
        else:
            df.at[index, 'Geocode result'] = 'Geocoding failed.'
            print(f"Index {index}: Unable to geocode company '{company_name}' using either name or address.")
            
        # Add a delay to respect API rate limits
        time.sleep(0.1)  # Adjust as needed based on your API quota

    return df